# Process training Data to train a RNN

In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals
import pathlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import pprint as pprint
import math
import time

In [2]:
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers

2.1.0


In [3]:
import tensorflow_docs as tfdocs
import tensorflow_docs.plots
import tensorflow_docs.modeling
import os
import json
import pickle
import sklearn
from sklearn import neighbors

### 1. Get the data from CSV files
First Import it using pandas

In [4]:
def class_to_action (class_int):
    with open('./data/class_to_action (28).json') as json_file:
        CtA = json.load(json_file)
        
    action = CtA[class_int]
    return action

In [13]:

def get_data (file):
    raw_dataset = pd.read_csv(file)
    dataset = raw_dataset.copy()
    training_size = len(dataset)
    N_CATEGORIES = 28

    dataset["A0"] = np.nan
    dataset["A1"] = np.nan

    for index, row in dataset.iterrows():
        CS_array =np.array(eval(row["current_state"]))
        dataset["current_state"][index] = CS_array
        GS_array =np.array(eval(row["goal_state"]))
        dataset["goal_state"][index] = GS_array
        dataset["A0"][index] = class_to_action(str(dataset["out_action"][index]))[0]
        dataset["A1"][index] = class_to_action(str(dataset["out_action"][index]))[1]

    # Usar estado final 
    dataset["Difference"] =   dataset["goal_state"]-dataset["current_state"]

    dataset[["C0",'C1',"C2","C3",'C4',"C5"]] = pd.DataFrame(dataset.current_state.values.tolist(), index= dataset.index)
    dataset[["D0",'D1',"D2","D3",'D4',"D5"]] = pd.DataFrame(dataset.Difference.values.tolist(), index= dataset.index)
    
    dataset_short= dataset[["id_camino","D0",'D1',"D2","D3",'D4',"D5","out_action","A0","A1"]]

    return dataset_short

# Normalize the data

It is good practice to normalize features that use different scales and ranges. Although the model *might* converge without feature normalization, it makes training more difficult, and it makes the resulting model dependent on the choice of units used in the input.

Note: Although we intentionally generate these statistics from only the training dataset, these statistics will also be used to normalize the test dataset. We need to do that to project the test dataset into the same distribution that the model has been trained on.

In [10]:
def norm(x, train_stats):
  return (x - train_stats['mean']) / train_stats['std']




In [12]:
def normalize_data(dataset_short,action_vector):
    reduced_dataset = dataset_short.copy()
    reduced_dataset.pop("out_action")
    reduced_dataset.pop("A0")
    reduced_dataset.pop("A1")
    reduced_dataset.pop("id_camino")
    data_stats = reduced_dataset.describe()
    data_stats = data_stats.transpose()

    data_stats.to_csv(r"./data/data_stats.csv")

    action_vector.append("id_camino")
    normed_data = norm(reduced_dataset, data_stats)
    normed_dataset = normed_data.join(dataset_short[action_vector])

    return normed_dataset

## Pad the data

In [14]:
def post_padding(normed_dataset,N_STEPS):
    PAD_LEN = N_STEPS+1
    padded_data = []
    for camino in normed_dataset["id_camino"]:
        group = normed_dataset[normed_dataset["id_camino"]==camino]
        group.pop("id_camino")
        padded_data.append(group.values)
    
    dataset2 = tf.keras.preprocessing.sequence.pad_sequences(padded_data, maxlen=PAD_LEN, dtype='float64', padding='post', truncating='post', value=0.0)

    return dataset2

This normalized data is what we will use to train the model.

Caution: The statistics used to normalize the inputs here (mean and standard deviation) need to be applied to any other data that is fed to the model.  That includes the test set as well as live data when the model is used in production.